# CISC 873 Data Mining Competition #2
name: Asmaa Qindeel
code: 21aaae1


Competition #2: https://www.kaggle.com/c/cisc-873-dm-f22-a2/overview

# TOPICS:
* [Questions & Answers](#Questions)
* [Problem and Protocol](#intro)

**Code Part**
* [Preparing workspace: gathering info about the data](#pre)
    * [Data Balance](#balance)
* [Trail 1: Grid Search & RandomForest](#t1)
* [Trail 2: Grid Search & RandomForest](#t2)
* [Trail 3: Random Search & Logistic Regression](#t3)
* [Trail 4: Random Search & ExtraTrees](#t4)
* [Trail 5: Random Search with XGboost](#t5)
* [Trail 6: Bayes Search & XGBoost with Data Resampling](#t6)
* [Trail 7: Bayes Search & SVC with Data Resampling](#t7)

[**Conclusion**](#conc)

# Questions:  <a class="anchor" id="Questions"></a>

1. Why a simple linear regression model (without any activation function) is not good for classification task, compared to Perceptron/Logistic regression?
    - A linear regressor will only inform you of a continous decision boundary(will not allow you to know how much probalbe this label is, how close to the boundary), while logestic regressor will inform you of the probability of this classification(discrete output). It also projects our input into another learning space, exploring more information.

2. What's a decision tree and how it is different to a logistic regression model?
    - decision tree is a branching technique for making decisions, it builds up desicions subsequent to each other. it is different to log reg in handling non linearly separable data, by dividing the feature space into smaller spaces.
3. What's the difference between grid search and random search?
    - grid will take longer time to search over all grid points(hyper sets), random will go over certain number of search points(iterations)
4. What's the difference between bayesian search and random search?
    - random iterates over grid points randomly and independantly in a certain number of iterations, while bayes builds up the search, by chosing what's promising in the previous search points into the next search point. making it (bayes) faster and more optimum.



## Refrences:
refrences used in this project include the following:

https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/

https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/

https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

The CISC_873_auto_tune lab

# Problem Formulation:<a class="anchor" id="intro"></a>

https://www.kaggle.com/c/cisc-873-dm-f22-a2/overview

The problem is a Binary classification, the input space is speed-dating information, the target is to predict wheather date is successfull or not(wheather it may lead to another date or not). Class 1 mean successful date, class_0 mean not a successful match/date. Through out this project i'll explore various models and tune their hyperparameter set with various methods. The object is to learn more about hyper-tuning and use pipelines to preprocess and train the model all at one stream. The ideal solution for a data like this lies in Trees models, because they use branching, which is the most suitable for this type of data.

Challenges as usual will be in the data, it has many inputs, very thorough, and being human collected thorough-survey means ==>> null data. It is also unbalanced as mentioned in the competitions overview. Another tuning i plan to do is to resample the data to test its effect over the result, i'll make the resampling technique itself a hyperparameter.

Metric used in this problem is roc_auc.

## The Impact:
for Me, learning. for the Dating company, more insight of the people they try to match together, so more successfull waves, and less disappointed customers.



# Experimental Protocol:

Using pipeline to contain the preprocessors and feature selector and my model.

Change models and search spaces. I also added Resamplers as a hyperparameter to find the best approach between; Under-sampling, Over-sampling, Combination of both.

I also use cross validation in grid search by two methods; automatic /in-function cross validation, with cv=2 to fit the grid twice, and Pre-defined split cross validation, with split ratio of 8/2 and random state=42.

For each Trial i try using the best hypers from the previous trial if possible, to shorten the search time. Otherwise, I will re-tune it.

In [1]:
# import basic liberary
import numpy as np
import pandas as pd

#import classifier
from skopt.space import Real, Categorical, Integer #tools for Bayes search
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier , ExtraTreesClassifier

#import preprocessors
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

# Install imbalanced-learn to resample data
# # pip install imbalanced-learn

#import imbalanced-learn.over_sampling, it contains the sampler classes like (RandomOverSampler,
                    #RandomUnderSampler, SMOTE, SMOTEN, BorderlineSMOTE)
import imblearn.over_sampling as over_sampling 
import imblearn.under_sampling as under_sampling
import imblearn.combine as combine

#import selection methods
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV, PredefinedSplit
from skopt import BayesSearchCV

#import evaluation metrics
from sklearn.metrics import roc_auc_score

#import pipelines lib
# make pipeline with imbalanced-learn lib to include the resampling models(Random under sampler, Smote)
from imblearn.pipeline import Pipeline as imb_pipe, make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# make the random numbers predictable, so the result doesn't change every time i run
np.random.seed(0) 

## Data Preparation <a class="anchor" id="pre"></a>

In [2]:
# Speed Dating dataset
# https://www.kaggle.com/c/cisc-873-dm-f22-a2/data

#load the two files
train_file = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CISC 873 Data Mining/C2/train.csv", header = 0)
test_file = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CISC 873 Data Mining/C2/train.csv', header =0)

# keep the original copy

df = train_file.copy()
df

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3,id
0,0,3,2,14,18,2,2.0,14,12,372.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2583
1,1,14,1,3,10,2,NaN,8,8,63.0,...,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,6830
2,1,14,1,13,10,8,8.0,10,10,331.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4840
3,1,38,2,9,20,18,13.0,6,7,200.0,...,9.0,8.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,5508
4,1,24,2,14,20,6,6.0,20,17,357.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5904,0,1,2,9,20,2,2.0,18,1,214.0,...,12.0,12.0,9.0,12.0,NaN,NaN,NaN,NaN,NaN,3390
5905,1,24,2,9,20,19,15.0,5,6,199.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4130
5906,0,13,2,11,21,5,5.0,3,18,290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1178
5907,1,10,2,7,16,6,14.0,9,10,151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5016


In [3]:
# # checking data:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5909 entries, 0 to 5908
Columns: 192 entries, gender to id
dtypes: float64(173), int64(11), object(8)
memory usage: 8.7+ MB


In [4]:
# Check for duplications
df.duplicated().sum()

0

In [5]:
# Check how much is missing
null_count = df.isnull().sum().sort_values(ascending = False)
null_count

num_in_3    5449
numdat_3    4849
expnum      4627
amb7_2      4519
sinc7_2     4519
            ... 
position       0
round          0
wave           0
condtn         0
id             0
Length: 192, dtype: int64

In [6]:
# categorical encoding of certain features, chosen from the data description file
df[['gender', 'condtn', 'samerace', 'field_cd', 'race', 'goal',
      'date', 'go_out', 'career_c', 'length', 'numdat_2', 'date_3']] = df[['gender', 
            'condtn', 'samerace', 'field_cd', 'race',  'goal', 'date', 
                  'go_out', 'career_c', 'length', 'numdat_2', 'date_3']].astype("category")

df.shape

(5909, 192)

In [7]:
#do the same for test file

#make a copy first
df_test = test_file.copy()

# categorical encoding of certain features:
df_test[['gender', 'condtn', 'samerace', 'field_cd', 'race', 'goal',
      'date', 'go_out', 'career_c', 'length', 'numdat_2', 'date_3']] = df_test[['gender', 
            'condtn', 'samerace', 'field_cd', 'race',  'goal', 'date', 
                  'go_out', 'career_c', 'length', 'numdat_2', 'date_3']].astype("category")

### Take a look at correlation:

In [8]:
import plotly.express as px
fig = px.imshow(df.corr())
fig.show() 

# Data Balance : <a class="anchor" id="balance"></a>
In the competitoin it was mentioned that the data was unbalanced, lets see how much


In [9]:
#checking balance of classes 0 & 1
print ( "class 0 samples:", (df.match == 0).sum())
print ( "class 1 samples:", (df.match == 1).sum())
print ( "ration of class_1/class_0:", (df.match == 1).sum()/ (df.match == 0).sum())

class 0 samples: 4921
class 1 samples: 988
ration of class_1/class_0: 0.20077220077220076


# Trial 1: Grid search & RandomForest  <a class="anchor" id="t1"></a>

**Thoughts**

for a first trial i will use the data with no preprocessing, I will not manually drop any columns, but i will use feature selection (select kbest) because the number of features is huge(192) for a problem like this, and most of the columns are too specific that the people filling the information got bored so most of it are more than half empty as shown before. I will use Simple Imputer to handle all the missing values without any effect over the samples. 

I will also encode the categorical features with OneHotEndoding, because it is already labeled as stated in the data description file, e.g.(field_cd: field coded, career_c: career coded)

**Plan:**
- split the numerical and categorical features for auto imputing
- Create Pipeline to contain: imputer, feature selection(kbest) and my Model
- Auto tune the model with grid_search and cross validation
- model: RandomForest
- predict test file data and save result

In [10]:
# now we can split the features and target 
y = df['match'] # lower case for vector
X = df.drop('match', axis=1) # upper case for matrix

In [11]:
# get the numerical column names and categorical column names for the encoding part

# categorical features 
features_categorical = X.select_dtypes(include=['category']).keys().to_list()
# numeric features 
features_numeric = X.select_dtypes(include=['float64', 'int64']).keys().to_list()

In [12]:
# define a pipeline for handling the numeric data
numeric_pipe = Pipeline(
    steps=[ # (name, model)
        ('imputer', SimpleImputer())]
)

# define a pipeline for categorical data
cat_pipe = Pipeline(
    steps=[ ('imputer', SimpleImputer()),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

#make two steps for the pipeline
    ## step 1 preprocessing data with ColumnTransformer
    ## step 2 learning: feature selection and model

# use column transformer because the pipeline can't handle the Imputers directly
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, features_numeric),
        ('cat', cat_pipe, features_categorical)
        
    ]
)

## step 2: add preprocessor to feature selection and model in pipe 
# the name is used later to process hyperparameters in each object
full_pipline = Pipeline(
    steps=[       
        ('preprocessor', preprocessor),
         ('kbest', SelectKBest(mutual_info_classif)), #
        ('my_classifier', RandomForestClassifier())
    ]
)

In [13]:
#define parameter space for the grid search
param_grid = {
    # strategy for the imputer
    'preprocessor__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
    #hypers of the RandomForest model  
    'my_classifier__n_estimators':[50, 70],
    'my_classifier__max_depth':[7, 10],
    
    #feature number for the feature selector (kbest)
    'kbest__k': [30,50], #prime numbers*10 # because i like prime numbers
}

# use 3 folds in cross-validation
# use 'roc_auc' for scoring

grid_search = GridSearchCV(
    full_pipline, param_grid, cv=2, verbose=1, 
    scoring='roc_auc')

#train the model over the entire data set
grid_search.fit(X, y)

print('best score of the cv:', grid_search.best_score_)
print('best Hyper set:', grid_search.best_params_)

Fitting 2 folds for each of 24 candidates, totalling 48 fits
best score of the cv: 0.8600473510659511
best Hyper set: {'kbest__k': 50, 'my_classifier__max_depth': 10, 'my_classifier__n_estimators': 70, 'preprocessor__num__imputer__strategy': 'median'}


In [14]:
#predict for the test file 
#and save to file
y_out = grid_search.predict_proba(test_file)

dummy = pd.DataFrame({'id': test_file['id'],'match': y_out[:,1]})
dummy.to_csv("grid_search_T1.csv", index = False)

<img src="https://imgur.com/b3Xpd6j.png" alt="drawing" width="300" hight="300"/>

I spent 5 days sending the predict() output, i thought i was overfitting. My overthinking projected in my code, it became a labyrinth, so I had to delete it all and restart over

# Trial 2: Grid Search & RandomForest <a class="anchor" id="t2"></a>

**Thoughts and observations:** 
Trail 1 with score 0.86 is pretty good, considering that the data is unbalanced, mostly NAN, no normalization, no preprocessing except simple imputer, i didn't even remove the id columns. actually this is better than expected, that i ran it several times to make sure.

for the second trial i will add some preprocessing, because i think different data means different results. also to try and explore other search spaces. for instance; 

- manually drop columns that are more than half empty, this should do nothing with the kbest model in place, but lets try anyway. also drop IDs columns(from the description file), and two columns(career, field) i noticed they are Encodel as labels in (career_c, field_c).
- normalize with StandardScalar, 
- and change the SimpleImputer to KNNImputer.

- Hypers chosen from last Trial: RandomForest__max depth = 7 , selectKBest__k_best = 50
- model: RandomforestTrees
- Tuning: grid_search with cross validation

In [15]:
# # get columns that are more than half empty
null_coulmns =  null_count[null_count > df.shape[0]/2 ].keys().to_series()

# #get identification columns (useless features) from 
id_columns = pd.Series(['idg',  'wave', 'idg','condtn','wave','round',
                'position', 'positin1','order','partner','pid', 'career', 'field']) #since there is career_c
column_2_drop = pd.concat([ null_coulmns, id_columns])
# drop ids columns (useless features in matching)
df2 = df.copy()
df2 = df2.drop(column_2_drop, axis =1)

# extract the data from the dataframe
y = df2['match'] # lower case for vector
X = df2.drop('match', axis=1) # upper case for matrix

In [16]:
# get the numerical column names and categorical column names for encoding later
# categorical features 
features_categorical = X.select_dtypes(include=['category']).keys().to_list()
# numeric features 
features_numeric = X.select_dtypes(include=['float64', 'int64']).keys().to_list()

# define a pipeline for handling the numeric data
numeric_pipe = Pipeline(
    steps=[ # (name, model)
        ('imputer', KNNImputer()), ### changed
     ('scaler', StandardScaler())] ### added
)

# define a pipeline for categorical data
cat_pipe = Pipeline(
    steps=[
        ('imputer', KNNImputer()), ### changed
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

#make two steps for the pipeline
    ## step 1 preprocessing data with ColumnTransformer
    ## step 2 learning: feature selection and model

# column transformer because the pipeline raises error at the imputer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, features_numeric),
        ('cat', cat_pipe, features_categorical)
         ]
)

## step 2: add preprocessor to feature selection and model in pipe 

full_pipline = Pipeline(
    steps=[       
        ('preprocessor', preprocessor),
         ('kbest', SelectKBest(mutual_info_classif)), #
        ('my_classifier', RandomForestClassifier())
    ]
)

In [17]:
#define parameter space for the grid search
param_grid = {
    #hypers of KNN Imputer: the K
    'preprocessor__num__imputer__n_neighbors': [5,7],  
     #hypers of the RandomForest model : n_estimators , max_depth 
    'my_classifier__n_estimators':[50, 70, 110],
    'my_classifier__max_depth':[7], #best value, chosen in previous Trials

    #feature number for the feature selector (kbest)
    'kbest__k': [50] #best value, chosen in previous Trials 
}

# use 2 folds in cross-validation
# use 'roc_auc' for scoring
grid_search = GridSearchCV(
    full_pipline, param_grid, cv=2, verbose=1, 
    scoring='roc_auc')

#train the model
grid_search.fit(X, y)

print('best score of the cv:', grid_search.best_score_)
print('best Hyper set:', grid_search.best_params_)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
best score of the cv: 0.8558214859955258
best Hyper set: {'kbest__k': 50, 'my_classifier__max_depth': 7, 'my_classifier__n_estimators': 110, 'preprocessor__num__imputer__n_neighbors': 7}


In [18]:
#predict for the test file data
#and save to file
y_out = grid_search.predict_proba(df_test)

dummy = pd.DataFrame({'id': df_test['id'],'match': y_out[:,1]})
dummy.to_csv("grid_search_T2.csv", index = False)

Result is lower by 0.001 ?! why?! because of the KNNImputer

<img src="https://i.kym-cdn.com/entries/icons/facebook/000/035/685/hmmm.jpg" alt="drawing" width="300" hight="300"/>

# Trial 3: Grid Search & Logestic Regression <a class="anchor" id="t3"></a>

**Thoughts and observations:** 
No improvments were added because of Trial_2 changes, so this is not the direction to go, in the next Trail i'll change the model and reverse preprocessing back to Trail_1 settings.

- use only simple imputer, keep StandardScalaer for the logisitic to not explode(easier calculations)

**changes:**
- model: Logestic Regression
- Tuning: grid_search with cross validation

In [19]:
# extract the data from the dataframe
y = df['match'] # lower case for vector
X = df.drop('match', axis=1) # upper case for matrix

In [20]:
# get the numerical column names and categorical column names for encoding later
# categorical features 
features_categorical = X.select_dtypes(include=['category']).keys().to_list()
# numeric features 
features_numeric = X.select_dtypes(include=['float64', 'int64']).keys().to_list()

# define a pipeline for handling the numeric data
numeric_pipe = Pipeline(
    steps=[ # (name, model)
        ('imputer', SimpleImputer()), ### changed
     ('scaler', StandardScaler())]  
)

# define a pipeline for categorical data
cat_pipe = Pipeline(
    steps=[
        ('imputer', SimpleImputer()), ### changed
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

#make two steps for the pipeline
    ## step 1 preprocessing data with ColumnTransformer
    ## step 2 learning: feature selection and model

# column transformer because the pipeline raises error at the imputer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, features_numeric),
        ('cat', cat_pipe, features_categorical)
         ]
)

## step 2: add preprocessor to feature selection and model in pipe 

full_pipline = Pipeline(
    steps=[       
        ('preprocessor', preprocessor),
         ('kbest', SelectKBest(mutual_info_classif)), #
        ('my_classifier', LogisticRegression())
    ]
)

In [21]:
#define parameter space for the grid search
param_grid = {
    #hypers of KNN Imputer: the K
    'preprocessor__num__imputer__strategy': ['mean'],  #chosen best last Trial
     #hypers of the Logistic Reg
    'my_classifier__penalty': [None, 'l1', 'l2'],
    'my_classifier__C':[1.0, 0.8],
    'my_classifier__max_iter': [10000], # so the model can converge
    #feature number for the feature selector (kbest)
    'kbest__k': [30,50] #prime numbers*10 # because i like prime numbers
}

# use 2 folds in cross-validation
# use 'roc_auc' for scoring
grid_search = GridSearchCV(
    full_pipline, param_grid, cv=2, verbose=1, 
    scoring='roc_auc')

#train the model
grid_search.fit(X, y)

print('best score of the cv:', grid_search.best_score_)
print('best Hyper set:', grid_search.best_params_)

Fitting 2 folds for each of 12 candidates, totalling 24 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning:


16 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  Fil

best score of the cv: 0.856088552232108
best Hyper set: {'kbest__k': 50, 'my_classifier__C': 0.8, 'my_classifier__max_iter': 10000, 'my_classifier__penalty': 'l2', 'preprocessor__num__imputer__strategy': 'mean'}


In [22]:
#predict for the test file data
#and save to file
y_out = grid_search.predict_proba(df_test)

dummy = pd.DataFrame({'id': df_test['id'],'match': y_out[:,1]})
dummy.to_csv("grid_search_T3.csv", index = False)

# Trial 4 Random Search & ExtraTrees <a class="anchor" id="t4"></a>

**thoughts and observations**
Results are not getting better, why woult they?! i guess it will be the same from now on, so in the next trial i'll try exploring more models, and tuning methods.

for this one I'll use `Random_search` with predefined validation set

**what's new** 
- Imputer: SimpleImputer with mean strategy. 
- StandardScalar
- model: ExtraTreesClassifier, it choses its spilts randomly. so lets try it with random search.
- tuning: Random Search with predifined validation set

In [38]:
# Same X,y from trail 2
print(X.shape, df_test.shape)

(5909, 191) (5909, 192)


In [24]:
# get the numerical column names and categorical column names for encoding later

# categorical features 
features_categorical = X.select_dtypes(include=['category']).keys().to_list()
# numeric features 
features_numeric = X.select_dtypes(include=['float64', 'int64']).keys().to_list()

# define a pipeline for handling the numeric data
numeric_pipe = Pipeline(
    steps=[ # (name, model)
        ('imputer', SimpleImputer()),
     ('scaler', StandardScaler())]
)

# define a pipeline for categorical data
cat_pipe = Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

#make two steps for the pipeline
    ## step 1 preprocessing data with ColumnTransformer
    ## step 2 learning: feature selection and model

# column transformer because the pipeline raises error at the imputer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, features_numeric),
        ('cat', cat_pipe, features_categorical)
         ]
)

## step 2: add preprocessor to feature selection and model in pipe 
# we gave them a name so we can set their hyperparameters
full_pipline = Pipeline(
    steps=[       
        ('preprocessor', preprocessor),
         ('kbest', SelectKBest(mutual_info_classif)), #
        ('my_classifier', ExtraTreesClassifier())
    ]
)

In [25]:
#define parameter space for the random search
param_grid = {   
     #hypers of the ExtraTrees model : n_estimators , max_depth 
    'my_classifier__n_estimators':[50, 70, 110],
    'my_classifier__max_depth':[7, 10, 13],
    #feature number for the feature selector kbest
    'kbest__k': [50 ] # the best chosen from last Trail
}

# Further split the original training set to a train and a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X, y , train_size = 0.8, stratify = y, random_state = 42)

# Create a list where train data indices are -1 and validation data indices are 0
split_index = [-1 if x in X_train.index else 0 for x in X.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

# create a random search
random_search = RandomizedSearchCV(
    full_pipline, param_grid, cv=pds, verbose=1,
    # number of random trials
    n_iter=20,
    scoring='roc_auc')

random_search.fit(X, y)

print('best score of the cv:', random_search.best_score_)
print('best Hyper set:', random_search.best_params_)

Fitting 1 folds for each of 9 candidates, totalling 9 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning:

The total space of parameters 9 is smaller than n_iter=20. Running 9 iterations. For exhaustive searches, use GridSearchCV.



best score of the cv: 0.8860967397552763
best Hyper set: {'my_classifier__n_estimators': 70, 'my_classifier__max_depth': 13, 'kbest__k': 50}


In [26]:
#predict for the test file data
# and save to file
y_out = random_search.predict_proba(df_test)

dummy = pd.DataFrame({'id': df_test['id'],'match': y_out[:,1]})
dummy.to_csv("random_search_T4.csv", index = False)

# Trial 5: Random Search with XGboost <a class="anchor" id="t5"></a>
**thoughts and observations**

trial 4 got 0.8768, Upgrade, HOORAY,
-which is an upgrade from the previous trials- 

this means we are going the right way with a trees classifier, as opposed to previous trials 2,3, which gained nothing. So we will keep the new changes from trial 4. Meaning the next plan :

**Plan:**
- model: Change the model (ExtraTrees was a hit), XGBoost is another trees classifier, it is widly used, very known for best behaviour, it uses Gradient Boosting.
"XGBoost always gives more importance to functional space when reducing the cost of a model."

- tuning: keep Random Search with predifined validation set

In [27]:
# Data Preparation
# original X,y from original dataframe
y = df['match']
X = df.drop('match', axis = 1)

In [28]:
# get the numerical column names and categorical column names for encoding later

# categorical features 
features_categorical = X.select_dtypes(include=['category']).keys().to_list()
# numeric features 
features_numeric = X.select_dtypes(include=['float64', 'int64']).keys().to_list()

# define a pipeline for handling the numeric data
numeric_pipe = Pipeline(
    steps=[ # (name, model)
        ('imputer', SimpleImputer())
     ]
)

# define a pipeline for categorical data
cat_pipe = Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

#make two steps for the pipeline
    ## step 1 preprocessing data with ColumnTransformer
    ## step 2 learning: feature selection and model

# column transformer because the pipeline raises error at the imputer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipe, features_numeric),
        ('cat', cat_pipe, features_categorical)
         ]
)

## step 2: add preprocessor to feature selection and model in pipe 
# we gave them a name so we can set their hyperparameters
full_pipline = Pipeline(
    steps=[       
        ('preprocessor', preprocessor),
         ('kbest', SelectKBest(mutual_info_classif)), #
        ('my_classifier', XGBClassifier(use_label_encoder=False))
    ]
)

#to ignore some error that i don't really understand, that raises from the XGB
y = y.astype(int)

In [29]:
#define parameter space for the random search and XGboost parameters
param_grid = {   
    'my_classifier__n_estimators':[50, 70, 110],
    #maxdepth of the Trees Classifier
    'my_classifier__max_depth':[7, 11, 13],
    'my_classifier__objective':['binary:logistic'], #objective for our problem
    'my_classifier__learning_rate':[0.1,0.01],
    #feature number for the feature selector kbest
    'kbest__k': [50] #best chosen from previous trial
}

# Further split the original training set to a train and a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X, y , train_size = 0.8, stratify = y, random_state = 42)

# Create a list where train data indices are -1 and validation data indices are 0
split_index = [-1 if x in X_train.index else 0 for x in X.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

# create a random search
random_search = RandomizedSearchCV(
    full_pipline, param_grid, cv=pds, verbose=1,
    # number of random trials
    n_iter=20,
    scoring='roc_auc')

random_search.fit(X, y)

print('best score of the cv:', random_search.best_score_)
print('best Hyper set:', random_search.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning:

The total space of parameters 18 is smaller than n_iter=20. Running 18 iterations. For exhaustive searches, use GridSearchCV.



Fitting 1 folds for each of 18 candidates, totalling 18 fits
best score of the cv: 0.8972088773918042
best Hyper set: {'my_classifier__objective': 'binary:logistic', 'my_classifier__n_estimators': 110, 'my_classifier__max_depth': 7, 'my_classifier__learning_rate': 0.1, 'kbest__k': 50}


In [30]:
#predict for the test file data
# and save to file
y_out = random_search.predict_proba(test_file)

dummy = pd.DataFrame({'id': test_file['id'],'match': y_out[:,1]})
dummy.to_csv("random_search_T5.csv", index = False)

0.9 HORAAAAAAAAAAAAAAAAAAAAAAAY 

<img src="https://imgflip.com/s/meme/Grumpy-Cat-Happy.jpg" alt="drawing" width="300" hight="300"/>

Hypers set: with XGBoost
{'my_classifier__objective': 'binary:logistic', 'my_classifier__n_estimators': 70, 'my_classifier__max_depth': 11, 'my_classifier__learning_rate': 0.1, 'kbest__k': 70}

# Trial 6: Bayes Search & XGBoost with Data Resampling
<a class="anchor" id="t6"></a>
xgboost with bayes

**thoughts and observations**
I think we achieved our best result in Trail 5 with 0.9 roc_auc, with Trees model XGBoost, No model will perform better.

Now lets explore Bayes search for hyper tuning.
The thing about bayes is that it converges to the optimal values faster, building up on each point's best hyper value...

**Data Resampling**

I will add data resampling techniques as a tunable hyperparameter, so the Bayes search tells me whether it is better to oversample data or undersample or combine both. With strategy of (0.5), meaning the minority class(class_1) will have a ratio of 0.5 of the whole data set. 
I could add the resample strategy too as a hyperparameter, but that was many parameters to search for, it would take too long

**Plan:**
- Imputer: SimpleImputer, No standardScaler, onehotEncoding
- model: keep XGBoost 
- tuning: Bayes search with predefined validation set

In [31]:
## step 2: add preprocessor to feature selection and model in pipe 
#           of type imbalanced-learn.pipeline, 
##     because the sklearn pipe can't take the resamplesr object

full_pipline = imb_pipe(
    steps=[       
        ('preprocessor', preprocessor),
        ('resampler' , None ), # will be changed as a hyperparameter
        ('kbest', SelectKBest(mutual_info_classif)),
        ('my_classifier', XGBClassifier(use_label_encoder=False))
    ]
)

#to ignore some error that i don't really understand, raises from the XGB
y = y.astype(int)

In [32]:

param_grid = {
    # XGBoost parameters
    'my_classifier__eval_metric': ['logloss'],
    'my_classifier__n_estimators':[50, 70],
    'my_classifier__max_depth':[11,13],
    'my_classifier__objective':['binary:logistic'], #objective for our problem
    'my_classifier__learning_rate':[0.01, 0.1],
    
    #feature number for the feature selector kbest
    'kbest__k': [70], #best value chosen from previous trial
    
    #type of the resampler
    'resampler': [None, 
                  under_sampling.RandomUnderSampler(sampling_strategy=0.5), #under sampler
                  over_sampling.BorderlineSMOTE(sampling_strategy=0.5), #over sampler
                 over_sampling.SMOTEN(sampling_strategy=0.5), #over sampler
                  over_sampling.ADASYN(sampling_strategy=0.5), #over sampler
                  combine.SMOTEENN(sampling_strategy=0.5),  #combiner resampler
                  combine.SMOTETomek(sampling_strategy=0.5) #combiner resampler
                 ]
}

# Further split the original training set to a train and a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X, y , train_size = 0.8, stratify = y, random_state = 42)

# Create a list where train data indices are -1 and validation data indices are 0
split_index = [-1 if x in X_train.index else 0 for x in X.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

# define ranges for bayes search
bayes_search = BayesSearchCV(
    full_pipline, param_grid,
    n_iter=5,
    random_state=0,
    scoring='roc_auc',
    verbose=1,
    # we still use 
    cv=pds
    
)
#train the model
bayes_search.fit(X, y)

#print results and best hyperparameters
print('best score of the cv:', bayes_search.best_score_)
print('best Hyper set:', bayes_search.best_params_)

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
best score of the cv: 0.8873029071199804
best Hyper set: OrderedDict([('kbest__k', 70), ('my_classifier__eval_metric', 'logloss'), ('my_classifier__learning_rate', 0.07553630702630122), ('my_classifier__max_depth', 12), ('my_classifier__n_estimators', 54), ('my_classifier__objective', 'binary:logistic'), ('resampler', None)])


0.88 roc_auc

Best Hyper set:


my_classifier__learning_rate'= 0.07553630702630122), 

my_classifier__max_depth= 12), 

my_classifier__n_estimators= 54)

'resampler'= None)

In [33]:
#predict for the test file data
# and save to file
y_out = bayes_search.predict_proba(test_file)

dummy = pd.DataFrame({'id': test_file['id'],'match': y_out[:,1]})
dummy.to_csv("bayes_search_T6.csv", index = False)

# Trial 7: Bayes Search with SVC 
<a class="anchor" id="t7"></a>
Trial_6 got 0.88

Not bad, and notice the resampler'= `None` was in the best set, meaning my problem DID NOT need resampling very much, that it scored 0.88 without it

I think we achieved our best result in Trail 5 with 0.9 roc_auc, with Trees model XGBoost and random search.

But lets explore one more model, SVC, it shouldn't be much different from the logestic regression, because they are both based on finding optimal decision boundary, unlike Tree models which keeps splitting the feature space into smaller spaces.

**Data Resampling**
I will add data resampling techniques as a tunable hyperparameter, so the Bayes search tells me whether it is better to oversample data or undersample or combine both. With strategy of (0.5), meaning the minority class(class_1) will have a ratio of 0.5 of the whole data set. 
I could add the strategy too as a hyperparameter, but that was many parameters to search for, it would take too long

**Plan:**
- Imputer: SimpleImputer, No standardScaler, onehotEncoding
- model:  SVC() 
- tuning: Bayes search with predefined validation set

In [34]:
# Data Preparation
# original X,y from original dataframe
y = df['match']
X = df.drop('match', axis = 1)

In [39]:
##               build on previous trail variables, e.g. preprocessor
## step 2: add preprocessor to feature selection and model in pipe 
#           of type imbalanced-learn.pipeline, 
##     because the sklearn pipe can't take the resamplesr

full_pipline = imb_pipe(
    steps=[       
        ('preprocessor', preprocessor),
        ('resampler' , None ),
        ('kbest', SelectKBest(mutual_info_classif)),
        ('my_classifier', SVC(probability=True))
    ]
)

In [40]:

param_grid = {
    #parameter space for the SVC model
    'my_classifier__C': Real(1e-6, 1e+6, prior='log-uniform'),
    'my_classifier__gamma': Real(1e-6, 1e+1, prior='log-uniform'),
    'my_classifier__degree': Integer(1,8),
    'my_classifier__kernel': Categorical(['linear', 'poly', 'rbf']),
    #feature number for the feature selector kbest
    'kbest__k': [70], #the optimal result of previous Trails
    
    #type of the resampler
    'resampler': [None, 
                  under_sampling.RandomUnderSampler(sampling_strategy=0.5), #under sampler
                  over_sampling.BorderlineSMOTE(sampling_strategy=0.5), #over sampler
                 over_sampling.SMOTEN(sampling_strategy=0.5), #over sampler
                  over_sampling.ADASYN(sampling_strategy=0.5), #over sampler
                  combine.SMOTEENN(sampling_strategy=0.5),  #combiner resampler
                  combine.SMOTETomek(sampling_strategy=0.5) #combiner resampler
                 ]
}

# Further split the original training set to a train and a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X, y , train_size = 0.8, stratify = y, random_state = 42)

# Create a list where train data indices are -1 and validation data indices are 0
split_index = [-1 if x in X_train.index else 0 for x in X.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

# define ranges for bayes search
bayes_search = BayesSearchCV(
    full_pipline, param_grid,
    n_iter=5,
    random_state=0,
    scoring='roc_auc',
    verbose=1,
    # we still use 
    cv=pds
    
)

bayes_search.fit(X, y)


print('best score of the cv:', bayes_search.best_score_)
print('best Hyper set:', bayes_search.best_params_)

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
best score of the cv: 0.8660692288741069
best Hyper set: OrderedDict([('kbest__k', 70), ('my_classifier__C', 0.0008318235767757963), ('my_classifier__degree', 6), ('my_classifier__gamma', 9.971554009280314e-05), ('my_classifier__kernel', 'linear'), ('resampler', BorderlineSMOTE(sampling_strategy=0.5))])


**Thoughts**
 
SCV with Bayes resulted roc_auc = 0.86

**Best Hyper set:** 

C', 0.0008318235767757963), 

('my_classifier__degree', 6), 

('my_classifier__gamma', 9.971554009280314e-05), 

('my_classifier__kernel', 'linear'), 

('resampler', `BorderlineSMOTE`(sampling_strategy=0.5))])


But, notice the resampling strategy: BorderlineSMOTE(sampling_strategy=0.5), This time actually over sampling had an effect over the outcome of the problem.
That is because oof the different search grids in each search. All things considered, the hyperparameter set behaves together, and needs to be investegated together, for each time you investigate the outcome could differ, Which does make sense. 

You are exposing your model to different insights each time your search a new grid.

In [41]:
#predict for the test file data
# and save to file
y_out = bayes_search.predict_proba(test_file)

dummy = pd.DataFrame({'id': test_file['id'],'match': y_out[:,1]})
dummy.to_csv("bayes_search_T7.csv", index = False)

## Conclusion 
<a class="anchor" id="conc"></a>

We made 7 Trails to see the impact of different hyperparameters, models, over the outcome of the problem.

Tree models performed highest, as expected, that is because of the nature of the data, it is discrete, categorical, sparce data, so it needed a branching technique, just like how we match make in real life, we search by branching the desicions on top of each other.

Another thing to notice is how much hyperparameters could be in one problem, every thing could be tuned, and each hyperparameter value will affect the final output one way or another(like in bayes). You are exposing your model to different insights each time your search a new grid.

One final thought is that grid search, while it takes longer time it will go over all possible sets of parameters, so it is better than random which just randomly search for certain number of iterations, but Bayes is different than both, it is unique in structure and output, and with prober setting it performs nicely.




